In [1]:
from engine.data import get_data_loader
from engine.lstm import LSTMCommandRecognition

import pytorch_lightning as pl
import torch

from torch import Tensor, nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from pytorch_lightning.callbacks import EarlyStopping

from engine.data import get_data_loader
from engine.metrics import accuracy
from engine.model_base import LightningBaseModule

torch.set_float32_matmul_precision("medium")

/home/dawid/miniconda3/envs/dl/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-13 09:32:33.194807: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-13 09:32:33.224125: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-13 09:32:33.927304: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not

In [2]:
train_loader = get_data_loader('train')
val_loader = get_data_loader('val')

In [3]:
for sample in train_loader:
    X, y = sample
    break

In [4]:
callbacks = [
    EarlyStopping(
        monitor="val_accuracy",
        mode="max",
        patience=10,
        min_delta=1e-4,
    )
]
model = LSTMCommandRecognition(80, 12, 128)
trainer = pl.Trainer(max_epochs=100, callbacks=callbacks)
trainer.fit(model, train_loader, val_loader)

/home/dawid/miniconda3/envs/dl/lib/python3.12/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params
---------------------------------------------
0 | lstm    | LSTM             | 107 K 
1 | linear  | Linear           | 1.5 K 
2 | softmax | Softmax          | 0     
3 | loss    | CrossEntropyLoss | 0     
---------------------------------------------
109 K     Trainable params
0         Non-trainable params
109 K     Total params
0.436     Total estimated model params size (MB)


/home/dawid/miniconda3/envs/dl/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/dawid/miniconda3/envs/dl/lib/python3.12/site-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/dawid/miniconda3/envs/dl/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 21: 100%|██████████| 837/837 [00:02<00:00, 366.91it/s, v_num=9, train_step_loss=1.980, val_step_loss=2.000]
